# day 18

https://adventofcode.com/2018/day/18

In [ ]:
import os

import eri.logging as logging

In [ ]:
FNAME = os.path.join('data', 'day18.txt')

LOGGER = logging.getLogger('day18')
logging.configure()

## part 1

### problem statement:

> On the outskirts of the North Pole base construction project, many Elves are collecting lumber.
> 
> The lumber collection area is 50 acres by 50 acres; each acre can be either open ground (.), trees (|), or a lumberyard (#). You take a scan of the area (your puzzle input).
> 
> Strange magic is at work here: each minute, the landscape looks entirely different. In exactly one minute, an open acre can fill with trees, a wooded acre can be converted to a lumberyard, or a lumberyard can be cleared to open ground (the lumber having been sent to other projects).
> 
> The change to each acre is based entirely on the contents of that acre as well as the number of open, wooded, or lumberyard acres adjacent to it at the start of each minute. Here, "adjacent" means any of the eight acres surrounding that acre. (Acres on the edges of the lumber collection area might have fewer than eight adjacent acres; the missing acres aren't counted.)
> 
> In particular:
> 
> + An open acre will become filled with trees if three or more adjacent acres contained trees. Otherwise, nothing happens.
> + An acre filled with trees will become a lumberyard if three or more adjacent acres were lumberyards. Otherwise, nothing happens.
> + An acre containing a lumberyard will remain a lumberyard if it was adjacent to at least one other lumberyard and at least one acre containing trees. Otherwise, it becomes open.
> 
> These changes happen across all acres simultaneously, each of them using the state of all acres at the beginning of the minute and changing to their new form by the end of that same minute. Changes that happen during the minute don't affect each other.
> 
> For example, suppose the lumber collection area is instead only 10 by 10 acres with this initial configuration:
> 
>     Initial state:
>     .#.#...|#.
>     .....#|##|
>     .|..|...#.
>     ..|#.....#
>     #.#|||#|#|
>     ...#.||...
>     .|....|...
>     ||...#|.#|
>     |.||||..|.
>     ...#.|..|.
>     
>     After 1 minute:
>     .......##.
>     ......|###
>     .|..|...#.
>     ..|#||...#
>     ..##||.|#|
>     ...#||||..
>     ||...|||..
>     |||||.||.|
>     ||||||||||
>     ....||..|.
>     
>     After 2 minutes:
>     .......#..
>     ......|#..
>     .|.|||....
>     ..##|||..#
>     ..###|||#|
>     ...#|||||.
>     |||||||||.
>     ||||||||||
>     ||||||||||
>     .|||||||||
>     
>     After 3 minutes:
>     .......#..
>     ....|||#..
>     .|.||||...
>     ..###|||.#
>     ...##|||#|
>     .||##|||||
>     ||||||||||
>     ||||||||||
>     ||||||||||
>     ||||||||||
>     
>     After 4 minutes:
>     .....|.#..
>     ...||||#..
>     .|.#||||..
>     ..###||||#
>     ...###||#|
>     |||##|||||
>     ||||||||||
>     ||||||||||
>     ||||||||||
>     ||||||||||
>     
>     After 5 minutes:
>     ....|||#..
>     ...||||#..
>     .|.##||||.
>     ..####|||#
>     .|.###||#|
>     |||###||||
>     ||||||||||
>     ||||||||||
>     ||||||||||
>     ||||||||||
>     
>     After 6 minutes:
>     ...||||#..
>     ...||||#..
>     .|.###|||.
>     ..#.##|||#
>     |||#.##|#|
>     |||###||||
>     ||||#|||||
>     ||||||||||
>     ||||||||||
>     ||||||||||
>     
>     After 7 minutes:
>     ...||||#..
>     ..||#|##..
>     .|.####||.
>     ||#..##||#
>     ||##.##|#|
>     |||####|||
>     |||###||||
>     ||||||||||
>     ||||||||||
>     ||||||||||
>     
>     After 8 minutes:
>     ..||||##..
>     ..|#####..
>     |||#####|.
>     ||#...##|#
>     ||##..###|
>     ||##.###||
>     |||####|||
>     ||||#|||||
>     ||||||||||
>     ||||||||||
>     
>     After 9 minutes:
>     ..||###...
>     .||#####..
>     ||##...##.
>     ||#....###
>     |##....##|
>     ||##..###|
>     ||######||
>     |||###||||
>     ||||||||||
>     ||||||||||
>     
>     After 10 minutes:
>     .||##.....
>     ||###.....
>     ||##......
>     |##.....##
>     |##.....##
>     |##....##|
>     ||##.####|
>     ||#####|||
>     ||||#|||||
>     ||||||||||
> 
> After 10 minutes, there are 37 wooded acres and 31 lumberyards. Multiplying the number of wooded acres by the number of lumberyards gives the total resource value after ten minutes: 37 * 31 = 1147.
> 
> What will the total resource value of the lumber collection area be after 10 minutes?

#### loading data

In [ ]:
def load_data(fname=FNAME):
    with open(fname) as fp:
        return fp.read()

In [ ]:
import numpy as np

def str_to_arr(s):
    return np.array([list(row.strip()) for row in s.strip().split('\n')])

def arr_to_str(a):
    return '\n'.join(
        ''.join(row)
        for row in a
    )

In [ ]:
test_data = """.#.#...|#.
.....#|##|
.|..|...#.
..|#.....#
#.#|||#|#|
...#.||...
.|....|...
||...#|.#|
|.||||..|.
...#.|..|."""
assert test_data == arr_to_str(str_to_arr(test_data))

#### function def

In [ ]:
OPEN = '.'
TREE = '|'
LUMBER = '#'

n2c = {0: OPEN, 1: TREE, 2: LUMBER}
c2n = {v: k for (k, v) in n2c.items()}

In [ ]:
def step(a0):
    a1 = a0.copy()
    for i in range(a0.shape[0]):
        i0 = max(0, i - 1)
        i1 = min(a0.shape[0], i + 2)
        for j in range(a0.shape[1]):
            j0 = max(0, j - 1)
            j1 = min(a0.shape[1], j + 2)
            window = a0[i0: i1, j0: j1]
            new_val = ctr_val = a0[i, j]
            if ctr_val == OPEN:
                if (window == TREE).sum() >= 3:
                    new_val = TREE
            elif ctr_val == TREE:
                if (window == LUMBER).sum() >= 3:
                    new_val = LUMBER
            elif ctr_val == LUMBER:
                if (window == LUMBER).sum() < 2 or (window == TREE).sum() < 1:
                    new_val = OPEN
            a1[i, j] = new_val
    return a1

In [ ]:
import tqdm

def q_1(data, num_steps=10):
    a = str_to_arr(data)
    for i in tqdm.tnrange(num_steps):
        a = step(a)
    num_tree = (a == TREE).sum()
    num_lumber = (a == LUMBER).sum()
    return num_tree * num_lumber

#### tests

some tests of the update step

In [ ]:
t_0 = str_to_arr(""".#.#...|#.
.....#|##|
.|..|...#.
..|#.....#
#.#|||#|#|
...#.||...
.|....|...
||...#|.#|
|.||||..|.
...#.|..|.""")

t_1 = str_to_arr(""".......##.
......|###
.|..|...#.
..|#||...#
..##||.|#|
...#||||..
||...|||..
|||||.||.|
||||||||||
....||..|.""")

t_2 = str_to_arr(""".......#..
......|#..
.|.|||....
..##|||..#
..###|||#|
...#|||||.
|||||||||.
||||||||||
||||||||||
.|||||||||""")

t_3 = str_to_arr(""".......#..
....|||#..
.|.||||...
..###|||.#
...##|||#|
.||##|||||
||||||||||
||||||||||
||||||||||
||||||||||""")

t_4 = str_to_arr(""".....|.#..
...||||#..
.|.#||||..
..###||||#
...###||#|
|||##|||||
||||||||||
||||||||||
||||||||||
||||||||||""")

t_5 = str_to_arr("""....|||#..
...||||#..
.|.##||||.
..####|||#
.|.###||#|
|||###||||
||||||||||
||||||||||
||||||||||
||||||||||""")

t_6 = str_to_arr("""...||||#..
...||||#..
.|.###|||.
..#.##|||#
|||#.##|#|
|||###||||
||||#|||||
||||||||||
||||||||||
||||||||||""")

t_7 = str_to_arr("""...||||#..
..||#|##..
.|.####||.
||#..##||#
||##.##|#|
|||####|||
|||###||||
||||||||||
||||||||||
||||||||||""")

t_8 = str_to_arr("""..||||##..
..|#####..
|||#####|.
||#...##|#
||##..###|
||##.###||
|||####|||
||||#|||||
||||||||||
||||||||||""")

t_9 = str_to_arr("""..||###...
.||#####..
||##...##.
||#....###
|##....##|
||##..###|
||######||
|||###||||
||||||||||
||||||||||""")

t_10 = str_to_arr(""".||##.....
||###.....
||##......
|##.....##
|##.....##
|##....##|
||##.####|
||#####|||
||||#|||||
||||||||||""")

In [ ]:
assert (step(t_0) == t_1).all()
assert (step(t_1) == t_2).all()
assert (step(t_2) == t_3).all()
assert (step(t_3) == t_4).all()
assert (step(t_4) == t_5).all()
assert (step(t_5) == t_6).all()
assert (step(t_6) == t_7).all()
assert (step(t_7) == t_8).all()
assert (step(t_8) == t_9).all()
assert (step(t_9) == t_10).all()

In [ ]:
def test_q_1():
    LOGGER.setLevel(logging.DEBUG)
    assert q_1(test_data) == 1147
    LOGGER.setLevel(logging.INFO)

In [ ]:
test_q_1()

#### answer

In [ ]:
q_1(load_data())

## part 2

### problem statement:

> This important natural resource will need to last for at least thousands of years. Are the Elves collecting this lumber sustainably?
> 
> What will the total resource value of the lumber collection area be after 1000000000 minutes?

#### function def

In [ ]:
def q_2(data):
    return q_1(data, num_steps=1000000000)

#### answer

In [ ]:
data = load_data()

z = []
a = str_to_arr(data)
for i in tqdm.tnrange(1, 1001):
    a = step(a)
    num_tree = (a == TREE).sum()
    num_lumber = (a == LUMBER).sum()
    z.append({
        'i': i,
        'val': num_tree * num_lumber,
        's': arr_to_str(a)
    })

In [ ]:
from collections import Counter

In [ ]:
c = Counter([_['val'] for _ in z])

In [ ]:
c.most_common(5)

In [ ]:
logging.getLogger('matplotlib').setLevel(logging.WARNING)
import matplotlib.pyplot as plt

In [ ]:
plt.plot(list(range(1000)), [_['val'] for _ in z])

In [ ]:
from ipywidgets import interact

@interact(i=(0, 1000))
def vis(i=0):
    print(z[i]['s'])

so we need to find the cycle and just calculate the values assocaited with each modulus

In [ ]:
z_half = z[len(z) // 2:]
Counter(_['val'] for _ in z_half)

In [ ]:
i0 = None
i1 = None
for el in z_half:
    if el['val'] == 195160:
        if i0 is None:
            i0 = el['i']
            continue
        elif i1 is None:
            i1 = el['i']
            break
i1 - i0

In [ ]:
modscore = {(_['i'] % 28, _['val']) for _ in z_half}
modscore

In [ ]:
modscore = {k: v for (k, v) in modscore}
modscore

In [ ]:
i = 1000000000
modscore[i % 28]

fin